**A Carteira 8Fundos DEF é composta por:**

  * _7.5% Fidelity Global Cnsmr Inds A-Dis-EUR LU0114721508_
  * _7.5% GS Glb SmCp CORE® Eq E EUR Acc Close LU0245181838_
  * _7.5% M&G Optimal Income Euro A-H Acc GB00B1VMCY93_
  * _27.5% PIMCO GIS Global Bond E Acc EUR H IE00B11XZ103_
  
  
  * _7.5% MFS® Meridian Global Equity A1 EUR LU0094560744_
  * _7.5% Jupiter Dynamic Bond L EUR Acc LU0853555380_
  * _7.5% Invesco Global Smaller Coms Eq E EUR Acc LU0607512935_
  * _27.5% PIMCO GIS GlbInvGrdCrdt E EUR Hdg Acc IE00B11XZ434_

**A Carteira 8Fundos EW é composta por:**

  * _12.5% Fidelity Global Cnsmr Inds A-Dis-EUR LU0114721508_
  * _12.5% GS Glb SmCp CORE® Eq E EUR Acc Close LU0245181838_
  * _12.5% M&G Optimal Income Euro A-H Acc GB00B1VMCY93_
  * _12.5% PIMCO GIS Global Bond E Acc EUR H IE00B11XZ103_
  
  
  * _12.5% MFS® Meridian Global Equity A1 EUR LU0094560744_
  * _12.5% Jupiter Dynamic Bond L EUR Acc LU0853555380_
  * _12.5% Invesco Global Smaller Coms Eq E EUR Acc LU0607512935_
  * _12.5% PIMCO GIS GlbInvGrdCrdt E EUR Hdg Acc IE00B11XZ434_

**A Carteira 8Fundos AGR é composta por:**

  * _20% Fidelity Global Cnsmr Inds A-Dis-EUR LU0114721508_
  * _20% GS Glb SmCp CORE® Eq E EUR Acc Close LU0245181838_
  * _5% M&G Optimal Income Euro A-H Acc GB00B1VMCY93_
  * _5% PIMCO GIS Global Bond E Acc EUR H IE00B11XZ103_
  
  
  * _20% MFS® Meridian Global Equity A1 EUR LU0094560744_
  * _20% Jupiter Dynamic Bond L EUR Acc LU0853555380_
  * _5% Invesco Global Smaller Coms Eq E EUR Acc LU0607512935_
  * _5% PIMCO GIS GlbInvGrdCrdt E EUR Hdg Acc IE00B11XZ434_

In [13]:
suppressMessages(library(PerformanceAnalytics))
suppressMessages(library(lubridate))
suppressMessages(library(quantmod))
suppressMessages(library(stringr))
suppressMessages(library(dplyr))
suppressMessages(library(SIT))

In [14]:
########### Carregar todas as bibliotecas necessárias ##########
options(warn=-1)

suppressMessages(library(PerformanceAnalytics))
suppressMessages(library(PortfolioAnalytics))
suppressMessages(library(quantmod))
suppressMessages(library(formattable))
suppressMessages(library(tseries))
suppressMessages(library(DT))

options(warn=0)
################## Carregar as minhas funções ##################
display <- function(dataframe) {
    print(rbind(head(dataframe, 3), tail(dataframe, 3)))
}

xts_to_dataframe <- function(dataframe) {
  data.frame(date=index(dataframe), coredata(dataframe))
}

dataframe_to_xts <- function(dataframe) {
  xts(data.frame(lapply(dataframe, as.numeric),
                 stringsAsFactors = FALSE),
      order.by = dataframe[, 1])[, -1]
}

quotes_to_numeric <- function(dataframe) {
  cbind(dataframe[1],
        data.frame(lapply(dataframe[-1], as.numeric)))
}

compute_returns.xts <- function(dataframe) {
  xts(apply(dataframe, 2, FUN = Return.calculate),
      order.by = index(dataframe))[-1 ,]
}

quotes_loader <- function(x, Begin="1999-12-31", End="2100-12-31", locf=TRUE,
                          normalize=TRUE, xts = FALSE, returns = FALSE) {

  # Import CSV
  Quotes <- read.csv('C:/Users/Luis/Google Drive/_GitHub/Carteiras_Clientes/Cotacoes_diarias.csv')
  
  # Date column as dates
  Quotes$Date <- as.Date(Quotes$Date, format = "%Y-%m-%d")
  
  # Select only desired funds
  Quotes <- Quotes[c('Date', x)]
  
  # Filtering Quotes by Dates
  Quotes <- na.omit(subset(Quotes, Date >= Begin & Date <= End))
  
  
  # Assigning Quotes to environment
  assign('Quotes', Quotes, envir = .GlobalEnv)
  
  if (xts) {
    Quotes.xts <- dataframe_to_xts(Quotes)
    # Assign Fundos.xts to environment
    assign('Quotes.xts', Quotes.xts, envir = .GlobalEnv)
    
  }
  
    # Create returns if TRUE
  if (returns) {
    # Get Returns dataframe
    Returns.xts <- compute_returns.xts(Quotes.xts)
    # Assign Fundos.xts to environment
    assign('Returns.xts', Returns.xts, envir = .GlobalEnv)
    
  }
}

In [15]:
ISINs  <- c('LU0114721508', 'LU0245181838', 'LU1670724373', "IE00B11XZ103", 'LU0094560744', 'LU0853555380', 'LU0607512935', 'IE00B11XZ434')
Nomes <- c('FCI', 'GS_SC', 'MG_OI', 'PimcoGB', 'MFS_GE', 'JupiterDB', 'InvescoSC', 'PimcoIG')

In [16]:
#### Begin_End #### 
From <- "2013-05-10" # Data de início
To   <- toString(today()) # Data de Fim

Begin_End <- paste(From, "/", To) # Data de início e fim juntos para xts

In [17]:
# Download dos fundos
quotes_loader(ISINs, xts=TRUE)

Fundos.xts = Quotes.xts

In [18]:
ls()

[1] "Begin_End"           "compute_returns.xts" "dataframe_to_xts"   
 [4] "display"             "From"                "Fundos.xts"         
 [7] "ISINs"               "Nomes"               "P1_weights"         
[10] "P2_weights"          "P3_weights"          "Quotes"             
[13] "Quotes.xts"          "quotes_loader"       "quotes_to_numeric"  
[16] "Returns"             "To"                  "xts_to_dataframe"

In [19]:
# Get Returns dataframe
Returns <- xts(apply(Fundos.xts, 2, FUN = Return.calculate),order.by = index(Fundos.xts))[-1 ,]

# Naming the columns in Returns
colnames(Returns) <- Nomes

In [20]:
### Portfolio Weights ####
P1_weights <- c(0.075, 0.075, 0.075, 0.275, 0.075, 0.075, 0.075, 0.275)
P2_weights <- c(0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125)
P3_weights <- c(0.20, 0.20, 0.05, 0.05, 0.20, 0.20, 0.05, 0.05)

In [27]:
#### Portfolios ####
Portfolio_1 <- Return.portfolio(Returns,
                                weights = P1_weights,
                                verbose = TRUE,
                                rebalance_on =  "years",
                                value = 100)

#### Portfolios ####
Portfolio_2 <- Return.portfolio(Returns,
                                weights = P2_weights,
                                verbose = TRUE,
                                rebalance_on =  "years",
                                value = 100)

#### Portfolios ####
Portfolio_3 <- Return.portfolio(Returns,
                                weights = P3_weights,
                                verbose = TRUE,
                                rebalance_on =  "years",
                                value = 100)

In [28]:
# Cálculos
Portfolio1 <- xts(round(rowSums(Portfolio_1$EOP.Value), 2), order.by = index(Fundos.xts[-1 ,]))
Portfolio2 <- xts(round(rowSums(Portfolio_2$EOP.Value), 2), order.by = index(Fundos.xts[-1 ,]))
Portfolio3 <- xts(round(rowSums(Portfolio_3$EOP.Value), 2), order.by = index(Fundos.xts[-1 ,]))

In [29]:
Portfolios <- cbind(Portfolio1, Portfolio2, Portfolio3)
colnames(Portfolios) <- c('8Fundos DEF', '8Fundos EW', '8Fundos AGR')

In [30]:
tail(Portfolios)

           8Fundos DEF 8Fundos EW 8Fundos AGR
2019-04-26      130.52     145.99      158.46
2019-04-29      130.49     146.03      158.56
2019-04-30      130.43     145.86      158.25
2019-05-01      130.48     145.92      158.34
2019-05-02      130.04     145.30      157.60
2019-05-03      130.33     145.72      158.15

In [31]:
# Exportar Portfolios 
write.csv(as.data.frame(Portfolios), 'Portfolios.csv')

In [32]:
Fundos.xts_monthly <- to.monthly(Fundos.xts$F0GBR04EAN)
Fundos.xts_monthly$F000000FH0 <- to.monthly(Fundos.xts$F000000FH0)
Fundos.xts_monthly$F0000007LD <- to.monthly(Fundos.xts$F0000007LD)
Fundos.xts_monthly$F0GBR06T61 <- to.monthly(Fundos.xts$F0GBR06T61)
Fundos.xts_monthly$F0GBR04E6R <- to.monthly(Fundos.xts$F0GBR04E6R)
Fundos.xts_monthly$F00000Q8QC <- to.monthly(Fundos.xts$F00000Q8QC)
Fundos.xts_monthly$F00000NGSG <- to.monthly(Fundos.xts$F00000NGSG)
Fundos.xts_monthly$F0GBR06T6F <- to.monthly(Fundos.xts$F0GBR06T6F)

ERROR: Error in try.xts(x): Error in UseMethod("as.xts") :   no applicable method for 'as.xts' applied to an object of class "NULL"


In [ ]:
dim(Fundos.xts_monthly)

In [ ]:
Fundos.xts_monthly <- Fundos.xts_monthly[, c(4, 8, 12, 16, 20, 24, 28, 32)]
colnames(Fundos.xts_monthly) <- c('F0GBR04EAN', 'F000000FH0', 'F0000007LD', 'F0GBR06T61',
                                  'F0GBR04E6R', 'F00000Q8QC', 'F00000NGSG', 'F0GBR06T6F')

In [ ]:
tail(Fundos.xts_monthly)

In [ ]:
plot.zoo(Fundos.xts_monthly)

In [ ]:
Returns_monthly <- ROC(Fundos.xts_monthly, n = 1, type = "discrete")[-1, ]

In [ ]:
head(Returns_monthly)

In [ ]:
#colnames(Returns) <- Nomes
Corr_Matrix <- round(cor(Returns),2)
Corr_Matrix

In [ ]:
# Exportar Matrix de correlação 
write.csv(as.data.frame(Corr_Matrix), 'Corr_Matrix.csv')

In [ ]:
Preturns <- xts(apply(Portfolios, 2, FUN = Return.calculate),order.by = index(Portfolios))[-1, ]

In [ ]:
head(Preturns)

In [ ]:
DD_DEF <- table.Drawdowns(Preturns[, 1])
DD_EW  <- table.Drawdowns(Preturns[, 2])
DD_AGR <- table.Drawdowns(Preturns[, 3])

In [ ]:
DD_DEF['Depth'] <- DD_DEF['Depth'] * 100
DD_EW ['Depth'] <- DD_EW ['Depth'] * 100
DD_AGR['Depth'] <- DD_AGR['Depth'] * 100

In [ ]:
DD_DEF

In [ ]:
DD_EW

In [ ]:
DD_AGR

In [ ]:
# Passar as tabelas de Drawdowns para .csv
write.csv(as.data.frame(DD_DEF), 'DD_DEF.csv')
write.csv(as.data.frame(DD_EW) , 'DD_EW.csv' )
write.csv(as.data.frame(DD_AGR), 'DD_AGR.csv')

In [ ]:
Nomes_fundos <- c('8Fundos DEF', '8Fundos EW', '8Fundos AGR')
Portfolios.df <- data.frame(Portfolios)
colnames(Portfolios.df) <- Nomes_fundos
rownames(Portfolios.df) <- ymd(rownames(Portfolios.df))

In [ ]:
tail(Portfolios.df)

In [ ]:
filter.df.date <- function (df, years=0, months=0, days=0, previous.row=FALSE)
{
  last_date <- ymd(rownames(tail(df, 1)))
  
  year.nr  <- year(last_date)
  month.nr <- month(last_date)
  day.nr   <- day(last_date)
  
  if (month.nr == 1 & months != 0){
    month.nr = 13
  }
  
  if (day.nr == 1 & days != 0 & month.nr %in% c(1, 3, 5, 7, 8, 10, 12)){
    day.nr = 31
    month.nr = month.nr - 1
  }
  
  if (day.nr == 1 & days != 0 & month.nr %in% c(4, 6, 9, 11)){
    day.nr = 32
    month.nr = month.nr - 1
  }
  
  if (day.nr == 1 & days != 0 & month.nr %in% c(2)){
    day.nr = 29
    month.nr = month.nr - 1
  }

  new_date <- ymd(str_replace_all(paste(year.nr - years,'-',
                                        month.nr - months, '-',
                                        day.nr - days),
                                  fixed(' '), ''))
  
  if (previous.row == TRUE){
    new.df <- rbind(tail(subset(df, rownames(df) < new_date), 1),
                    subset(df, rownames(df) >= new_date))
    return(new.df)
  }
  
  if (previous.row == FALSE){
    return(subset(df, rownames(df) >= new_date))
  }
}

In [ ]:
one_year.df  <- filter.df.date(Portfolios.df, years=1, previous.row=T)
one_year.xts <- xts(one_year.df, order.by = ymd(rownames(one_year.df)))
one_year.xts.ret <- xts(apply(one_year.xts, 2, FUN=Return.calculate), order.by = index(one_year.xts))[-1, ]

In [ ]:
head(one_year.df)

In [ ]:
cagr_one_year <- round((((tail(one_year.df, 1) / head(one_year.df, 1))) - 1) * 100, 2)
rownames(cagr_one_year) <- 'CAGR'

one_year_sd <- round(t(apply(one_year.xts.ret, 2, FUN=StdDev.annualized)) * 100, 2)
one_year_sharpe <- cagr_one_year / one_year_sd

In [ ]:
three_years.df  <- filter.df.date(Portfolios.df, years=3, previous.row=T)
three_years.xts <- xts(three_years.df, order.by = ymd(rownames(three_years.df)))
three_years.xts.ret <- xts(apply(three_years.xts, 2, FUN=Return.calculate), order.by = index(three_years.xts))[-1, ]

In [ ]:
head(three_years.df)

In [ ]:
cagr_three_years <- round((((tail(three_years.df, 1) / head(three_years.df, 1)) ^ (1/3)) - 1) * 100, 2)
rownames(cagr_three_years) <- 'CAGR'

three_years_sd <- round(t(apply(three_years.xts.ret, 2, FUN=StdDev.annualized)) * 100, 2)
three_years_sharpe <- cagr_three_years / three_years_sd

In [ ]:
five_years.df  <- filter.df.date(Portfolios.df, years=5, previous.row=T)
five_years.xts <- xts(five_years.df, order.by = ymd(rownames(five_years.df)))
five_years.xts.ret <- xts(apply(five_years.xts, 2, FUN=Return.calculate), order.by = index(five_years.xts))[-1, ]

cagr_five_years <- round((((tail(five_years.df, 1) / head(five_years.df, 1)) ^ (1/5)) - 1) * 100, 2)
rownames(cagr_five_years) <- 'CAGR'

five_years_sd <- round(t(apply(five_years.xts.ret, 2, FUN=StdDev.annualized)) * 100, 2)
five_years_sharpe <- cagr_five_years / five_years_sd

In [ ]:
head(five_years.df)

In [ ]:
ten_years.df  <- filter.df.date(Portfolios.df, years=10, previous.row=T)
ten_years.xts <- xts(ten_years.df, order.by = ymd(rownames(ten_years.df)))
ten_years.xts.ret <- xts(apply(ten_years.xts, 2, FUN=Return.calculate), order.by = index(ten_years.xts))[-1, ]

cagr_ten_years <- round((((tail(ten_years.df, 1) / head(ten_years.df, 1)) ^ (1/10)) - 1) * 100, 2)
rownames(cagr_ten_years) <- 'CAGR'

ten_years_sd <- round(t(apply(ten_years.xts.ret, 2, FUN=StdDev.annualized)) * 100, 2)
ten_years_sharpe <- cagr_ten_years / ten_years_sd

In [ ]:
head(ten_years.df)

In [ ]:
one_year_pt <- round(rbind(cagr_one_year, one_year_sd, one_year_sharpe), 2)
rownames(one_year_pt) <- c('CAGR', 'Std.dev', 'Sharpe')

In [ ]:
three_years_pt <- round(rbind(cagr_one_year, one_year_sd, one_year_sharpe), 2)
rownames(one_year_pt) <- c('CAGR', 'Std.dev', 'Sharpe')

In [ ]:
three_years_pt <- round(rbind(cagr_three_years, three_years_sd, three_years_sharpe), 2)
rownames(three_years_pt) <- c('CAGR', 'Std.dev', 'Sharpe')

In [ ]:
five_years_pt <- round(rbind(cagr_five_years, five_years_sd, five_years_sharpe), 2)
rownames(five_years_pt) <- c('CAGR', 'Std.dev', 'Sharpe')

In [ ]:
ten_years_pt <- round(rbind(cagr_ten_years, ten_years_sd, ten_years_sharpe), 2)
rownames(ten_years_pt) <- c('CAGR', 'Std.dev', 'Sharpe')

In [ ]:
# Passar as tabelas de performance para .csv
write.csv(as.data.frame(one_year_pt), 'one_year_pt.csv')
write.csv(as.data.frame(three_years_pt), 'three_years_pt.csv')
write.csv(as.data.frame(five_years_pt), 'five_years_pt.csv')
write.csv(as.data.frame(ten_years_pt), 'ten_years_pt.csv')

In [ ]:
one_year_rolling <- round(data.frame(ROC(Portfolios.df, n = 262, type = "discrete") * 100), 2)
colnames(one_year_rolling) <- Nomes_fundos

In [ ]:
tail(one_year_rolling)

In [ ]:
one_year_pt

In [ ]:
three_years_rolling <- data.frame(ROC(Portfolios.df, n = 261 * 3, type = "discrete"))
three_years_rolling <- round(((((1 + three_years_rolling) ^ (1/3)) - 1 ) * 100), 2)
colnames(three_years_rolling) <- Nomes_fundos

In [ ]:
tail(three_years_rolling)

In [ ]:
three_years_pt

In [ ]:
# write .csv of rolling returns per year
write.csv(as.data.frame(one_year_rolling), 'one_year_rolling.csv')
write.csv(as.data.frame(three_years_rolling), 'three_years_rolling.csv')

In [ ]:
def_rolling <- cbind(one_year_rolling[, 1], three_years_rolling[, 1])
years <- c('1 Ano', '3 Anos')
colnames(def_rolling) <- years

In [ ]:
tail(def_rolling)

In [ ]:
ew_rolling <- cbind(one_year_rolling[, 2], three_years_rolling[, 2])
colnames(ew_rolling) <- years

In [ ]:
agr_rolling <- cbind(one_year_rolling[, 3], three_years_rolling[, 3])
colnames(agr_rolling) <- years

In [ ]:
# write .csv of rolling returns per portfolio
write.csv(as.data.frame(def_rolling), 'def_rolling.csv')
write.csv(as.data.frame(ew_rolling),  'ew_rolling.csv')
write.csv(as.data.frame(agr_rolling), 'agr_rolling.csv')

In [ ]:
today <-ymd('2017-10-04')
dob   <-ymd('2007-04-23')

In [ ]:
as.numeric(difftime(today, dob, unit="weeks"))/52.25

In [ ]:
dim(one_year_rolling)

In [ ]:
2728 / 10.4360902255639

In [ ]:
class(one_year_rolling)